In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime,date,timedelta,timezone
import pytz
import json

from tensorflow.keras.models import load_model
import joblib

from google.cloud import storage
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest


In [ ]:
# uncomment the following section
# fuction
# json and env if/else
# return statment

# import functions_framework
# @functions_framework.http
# def forecast_asset_movement(request):


# Constant & Parameter Variable

In [ ]:
loadModelMode='local'   # local,gcs

#today='2023-05-31' # last record of training data to predict the first movement
today='2023-08-25'
model_id='spy-ema1-60t10-ds0115t0523'

input_sequence_length =60
output_sequence_length =10

# if request.get_json():
#     print("JSON Post Date Info") # Post Method
#     request_json = request.get_json()
#     today=request_json['TODAY']
#     model_id=request_json['MODEL_ID']

# else:
#     print("Enviroment Variable Info")
#     today=os.environ.get('TODAY', '') 
#     #today=os.environ.get('TODAY', '2023-04-28')  
#     model_id=os.environ.get('MODEL_ID', 'spy-ema1-60t10-ds0115t0523')  
 

print("List parameter as belows")
print(f"today={today}")
print(f"model_id={model_id}")



# BigQuery Setting

In [ ]:
projectId='pongthorn'
dataset_id='FinAssetForecast'
table_data_id = f"{projectId}.{dataset_id}.fin_data"
table_id = f"{projectId}.{dataset_id}.fin_movement_forecast"

table_model_id= f"{projectId}.{dataset_id}.model_ts_metadata"

print(table_id)
print(table_data_id)
print(table_model_id)

client = bigquery.Client(project=projectId )

def load_data_bq(sql:str):
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 return df


# Load Model  Configuration MetaData

In [5]:
sqlModelMt=f"""
SELECT * FROM `{table_model_id}`  where model_id='{model_id}'
"""
print(sqlModelMt)
dfModelMeta=load_data_bq(sqlModelMt)
if dfModelMeta.empty==False:
    modelMeta=dfModelMeta.iloc[0,:]
    asset_name=modelMeta['asset']
    prediction_name=modelMeta['prediction']
    
    input_sequence_length=int(modelMeta['input_sequence_length'])
    output_sequence_length=int(modelMeta['output_sequence_length'])
    
    model_path=modelMeta['gs_model_path']
    local_model_path=modelMeta['local_model_path']
    
    model_file=modelMeta['model_file']
    scaler_file=modelMeta['scaler_file']
    scalerPred_file=modelMeta['scaler_pred_file']
    print(modelMeta)
    
else: 
    raise Exception(f"Not found model id  {model_id}")





SELECT * FROM `pongthorn.FinAssetForecast.model_ts_metadata`  where model_id='spy-ema1-60t10-ds0115t0523'

model_id                                         spy-ema1-60t10-ds0115t0523
asset                                                                   SPY
prediction                                                             EMA1
input_sequence_length                                                    60
output_sequence_length                                                   10
gs_model_path                gs://demo-ts-forecast-pongthorn/model_spy_ema1
local_model_path                                       model/model_spy_ema1
model_file                         EMA1_60To10_SPY_E150S20-Y2015-2023_ma.h5
scaler_file                    scaler_EMA1_60To10_SPY_E150S20-Y2015-2023.gz
scaler_pred_file          scaler_pred_EMA1_60To10_SPY_E150S20-Y2015-2023.gz
Name: 0, dtype: object


# Load model and scaler

In [6]:
if loadModelMode=='local':
 objectPaht=local_model_path
else:
 objectPaht=model_path  

model_path=f"{objectPaht}/{model_file}"
scale_input_path=f"{objectPaht}/{scaler_file}"
scale_output_path=f"{objectPaht}/{scalerPred_file}"

print(f"load model from {loadModelMode}")   
print(model_path)
print(scale_input_path)
print(scale_output_path)



load model from local
model/model_spy_ema1/EMA1_60To10_SPY_E150S20-Y2015-2023_ma.h5
model/model_spy_ema1/scaler_EMA1_60To10_SPY_E150S20-Y2015-2023.gz
model/model_spy_ema1/scaler_pred_EMA1_60To10_SPY_E150S20-Y2015-2023.gz


In [ ]:
if loadModelMode=='local':
    try:
        print("Model and Scaler Object Summary")
        x_model = load_model(model_path)
 
    except Exception as ex:
        print(str(ex))
        raise Exception(str(ex)) 
    
    try:
        print("Scaler Max-Min")
        x_scaler = joblib.load(scale_input_path)
        x_scalerPred=joblib.load(scale_output_path)

    except Exception as ex:
        print(str(ex))
        raise Exception(str(ex))

    print("=====================================================================================================")

In [62]:
 
if loadModelMode=='gcs':
 try:
    gcs_client = storage.Client()

    with open(scaler_file, 'wb') as scaler_f, open(scalerPred_file, 'wb') as scaler_pred_f,open(model_file, 'wb') as model_f:
        gcs_client.download_blob_to_file(scale_input_path, scaler_f
        )
        gcs_client.download_blob_to_file(scale_output_path, scaler_pred_f
        )
        gcs_client.download_blob_to_file(model_path, model_f
        )

    x_scaler = joblib.load(scaler_file)
    x_scalerPred=joblib.load(scalerPred_file)
    x_model = load_model(model_file)
 except Exception as ex:
    print(str(ex))
    raise Exception(str(ex))


In [63]:
print(x_model.summary())
#(max - min) / (X.max(axis=0) - X.min(axis=0))
print(f"max={x_scaler.data_max_} and min={x_scaler.data_min_} and scale={x_scaler.scale_}")
print(f"max={x_scalerPred.data_max_} and min={x_scalerPred.data_min_} and scale={x_scalerPred.scale_}")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 180)               131040    
                                                                 
 dropout_1 (Dropout)         (None, 180)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1810      
                                                                 
Total params: 132,850
Trainable params: 132,850
Non-trainable params: 0
_________________________________________________________________
None
max=[473.99] and min=[187.09] and scale=[0.00348554]
max=[473.99] and min=[187.09] and scale=[0.00348554]


# Declare and Initialize TS Model Variable

In [64]:
date_col='Date'
prediction_col=prediction_name
feature_cols=[prediction_name]


nLastData=input_sequence_length*2

# dt_imported=datetime.now()
dt_imported=datetime.now(timezone.utc)
dtStr_imported=dt_imported.strftime("%Y-%m-%d %H:%M:%S")
print(dtStr_imported)


2023-07-05 08:02:26


# Query Fin Data from BQ

In [65]:
print(f"Check Last Record as starting point to retrive  sequence {prediction_col} over the past {input_sequence_length} day")

Check Last Record as starting point to retrive  sequence EMA1 over the past 60 day


In [66]:

lastDate=None
if today=='':
    sqlLastDate=f""" select max(Date) as LastDate  from `{table_data_id}` where Symbol='{asset_name}' """

else:
    sqlLastDate=f""" 
    select Date as LastDate  from `{table_data_id}` where Symbol='{asset_name}' 
    and Date='{today}'
    """
print(sqlLastDate)
results = client.query(sqlLastDate)
dfLastDate=results.to_dataframe()
print(dfLastDate)
if dfLastDate.empty:
    print( f"Not found price data at {today}  of {asset_name}")
    # return f"Not found price data at {today}  of {asset_name}"
else:
    lastDate=dfLastDate.iloc[0,0]
    today=lastDate.strftime('%Y-%m-%d')


print(f"Forecast {prediction_col} movement of  {asset_name} at {today}")


 
    select Date as LastDate  from `pongthorn.FinAssetForecast.fin_data` where Symbol='SPY' 
    and Date='2023-06-30'
    
     LastDate
0  2023-06-30
Forecast EMA1 movement of  SPY at 2023-06-30


In [67]:
print(f"Check whether {asset_name}-{prediction_col} as {today} was predicted the future for the next {output_sequence_length} days")
sqlLastPred=f"""select prediction_date,asset_name,prediction_name,pred_timestamp from `{table_id}` 
where prediction_date='{today}' and   asset_name='{asset_name}' and prediction_name='{prediction_col}'
order by pred_timestamp 
"""
print(sqlLastPred)
dfLastPred=load_data_bq(sqlLastPred)
if dfLastPred.empty==False:
   dfLastPred=dfLastPred.drop_duplicates(subset=['prediction_date','asset_name','prediction_name'],keep='last') 
   print(f"{asset_name}-{prediction_col}-{today} has been predicted price movement")
   print(dfLastPred)
   # return f"Prediction price movement of {asset_name}-{prediction_col} at {today} has been predicted"
else:
       print(f"{asset_name}-{prediction_col} at {today} has not been predicted price movement yet.") 
       print("The system is about to predict price movement shortly.") 
       print("=======================================================================================")

Check whether SPY-EMA1 as 2023-06-30 was predicted the future for the next 10 days
select prediction_date,asset_name,prediction_name,pred_timestamp from `pongthorn.FinAssetForecast.fin_movement_forecast` 
where prediction_date='2023-06-30' and   asset_name='SPY' and prediction_name='EMA1'
order by pred_timestamp 

SPY-EMA1 at 2023-06-30 has not been predicted price movement yet.


In [68]:
dayAgo=datetime.strptime(today,'%Y-%m-%d') +timedelta(days=-nLastData)
print(f"Get data from {dayAgo.strftime('%Y-%m-%d')} - {today} as input to forecast")

sql=f"""
SELECT  *  FROM `{table_data_id}`  
Where  {date_col} between  DATE_SUB(DATE '{today}', INTERVAL {nLastData} DAY) 
and '{today}' and Symbol='{asset_name}' order by {date_col},ImportDateTime
"""
print(sql)
query_result=client.query(sql)
df=query_result.to_dataframe()

df=df.drop_duplicates(subset=[date_col,'Symbol'],keep='last')
df[date_col]=pd.to_datetime(df[date_col],format='%Y-%m-%d')
df.set_index(date_col,inplace=True)

print(df.info())

print(df[['Symbol','Close' ,'ImportDateTime']].head())
print(df[['Symbol','Close' ,'ImportDateTime']].tail())

if df.empty==True or len(df)<input_sequence_length:
    print(f"There is no enough data to make prediction during {dayAgo.strftime('%Y-%m-%d')} - {today}")
    # return f"There is no enough data to make prediction during {dayAgo.strftime('%Y-%m-%d')} - {today}"

Get data from 2023-03-02 - 2023-06-30 as input to forecast

SELECT  *  FROM `pongthorn.FinAssetForecast.fin_data`  
Where  Date between  DATE_SUB(DATE '2023-06-30', INTERVAL 120 DAY) 
and '2023-06-30' and Symbol='SPY' order by Date,ImportDateTime

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 84 entries, 2023-03-02 to 2023-06-30
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Symbol          84 non-null     object        
 1   Close           84 non-null     float64       
 2   EMA1            84 non-null     float64       
 3   EMA2            84 non-null     float64       
 4   MACD            84 non-null     float64       
 5   SIGNAL          84 non-null     float64       
 6   ImportDateTime  84 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 5.2+ KB
None
           Symbol   Close             ImportDateTime
Date                             

In [69]:
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
# import seaborn as sns

# plt.subplots(2, 1, figsize = (20, 10),sharex=True)

# ax1 = plt.subplot(2, 1, 1)
# plt.plot(df[['Close','EMA1','EMA2']])
# plt.ylabel('Price & EMA')

# ax2 = plt.subplot(2, 1, 2)
# plt.plot(df[['MACD','SIGNAL']])
# plt.xlabel('Date')
# plt.ylabel('MACD & Signal')

# plt.show()

# Get only Feature( 1 Indicator) to Predict itself in the next N days

In [70]:
print(f"Get Feature to Predict : {prediction_col} ")
dfForPred=df[feature_cols]
#dfForPred=dfForPred.iloc[-(input_sequence_length+1):-1,:]
dfForPred=dfForPred.iloc[-input_sequence_length:,:]
print(dfForPred.info())
print(dfForPred.shape)

print(dfForPred.head(10))
print(dfForPred.tail(10))

# dfForPred.plot(figsize = (20, 10))
# plt.show()

Get Feature to Predict : EMA1 
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2023-04-05 to 2023-06-30
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   EMA1    60 non-null     float64
dtypes: float64(1)
memory usage: 960.0 bytes
None
(60, 1)
                EMA1
Date                
2023-04-05  403.7406
2023-04-06  404.7314
2023-04-10  405.6184
2023-04-11  406.3641
2023-04-12  406.6707
2023-04-13  407.9069
2023-04-14  408.7347
2023-04-17  409.6812
2023-04-18  410.5046
2023-04-19  411.1656
                EMA1
Date                
2023-06-16  433.6472
2023-06-20  434.2963
2023-06-21  434.4133
2023-06-22  434.7945
2023-06-23  434.5035
2023-06-26  433.9469
2023-06-27  434.3545
2023-06-28  434.7246
2023-06-29  435.3401
2023-06-30  436.7793


# Make Pediction as Forecast

In [71]:
xUnscaled=dfForPred.values #print(xUnscaled.shape)
xScaled=x_scaler.transform(xUnscaled)
print(xScaled.shape)
# print(xScaled[-5:])

# # Way1
# xScaledToPredict = []
# xScaledToPredict.append(xScaled)
# print(len(xScaledToPredict))

# yPredScaled=x_model.predict(np.array(xScaledToPredict))
# print(yPredScaled.shape,yPredScaled)

# yPred  = x_scalerPred.inverse_transform(yPredScaled.reshape(-1, 1))
# print(yPred.shape,yPred)

#Way2
xScaledToPredict= xScaled.reshape(1,input_sequence_length,len(feature_cols))
print(xScaledToPredict.shape)

yPredScaled = x_model.predict(xScaledToPredict)
print(yPredScaled.shape, yPredScaled)

yPred = x_scalerPred.inverse_transform(yPredScaled).reshape(-1, 1)
print(yPred.shape, yPred)


print("============================Summary============================")
print(xUnscaled.shape)
print(yPred.shape)

print("============================Input============================")
print(xUnscaled)
print("============================Output============================")
print(yPred)



(60, 1)
(1, 60, 1)
1/1 [==============================] - 1s 891ms/step
(1, 10) [[0.88172513 0.8705535  0.8742105  0.86745614 0.88415515 0.8758461
  0.87507915 0.88143235 0.8743186  0.8755618 ]]
(10, 1) [[440.05695]
 [436.8518 ]
 [437.901  ]
 [435.96317]
 [440.75412]
 [438.37024]
 [438.1502 ]
 [439.97293]
 [437.932  ]
 [438.28867]]
============================Summary============================
(60, 1)
(10, 1)
============================Input============================
[[403.7406]
 [404.7314]
 [405.6184]
 [406.3641]
 [406.6707]
 [407.9069]
 [408.7347]
 [409.6812]
 [410.5046]
 [411.1656]
 [411.2955]
 [411.4599]
 [411.6727]
 [410.6558]
 [409.5111]
 [410.0382]
 [411.1094]
 [411.9095]
 [411.7151]
 [411.0432]
 [409.9681]
 [410.4521]
 [410.8681]
 [410.8793]
 [411.2376]
 [411.3999]
 [411.4344]
 [411.7209]
 [411.4535]
 [412.1401]
 [413.4292]
 [414.373 ]
 [415.1761]
 [414.9786]
 [414.2716]
 [414.3404]
 [415.373 ]
 [416.247 ]
 [416.5385]
 [417.4988]
 [419.3935]
 [420.7947]
 [422.1102]
 [422.91

# Build Prediction Result Data

## Feature Data

In [72]:
print("Create indexes from Dataframe dfForPred")
dfFeature=pd.DataFrame(data= xUnscaled,columns=feature_cols,index=dfForPred.index)

print(dfFeature.shape)
print(dfFeature.head())
print(dfFeature.tail())

Create indexes from Dataframe dfForPred
(60, 1)
                EMA1
Date                
2023-04-05  403.7406
2023-04-06  404.7314
2023-04-10  405.6184
2023-04-11  406.3641
2023-04-12  406.6707
                EMA1
Date                
2023-06-26  433.9469
2023-06-27  434.3545
2023-06-28  434.7246
2023-06-29  435.3401
2023-06-30  436.7793


## Forecast/Prediction Value Data

In [73]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

print("Create indexes by specifying output_sequence_length stating from get last record of DFFeature+1")
lastRowOfFeature=dfFeature.index.max()
firstRowofPrediction=lastRowOfFeature+timedelta(days=1)
datePred=pd.date_range(start=firstRowofPrediction,freq=us_bd,periods=output_sequence_length)
print(datePred)

dfPrediction=pd.DataFrame(data= yPred,columns=feature_cols,index=datePred)
dfPrediction.index.name=date_col
print(dfPrediction.shape)
print(dfPrediction)


Create indexes by specifying output_sequence_length stating from get last record of DFFeature+1
DatetimeIndex(['2023-07-03', '2023-07-05', '2023-07-06', '2023-07-07',
               '2023-07-10', '2023-07-11', '2023-07-12', '2023-07-13',
               '2023-07-14', '2023-07-17'],
              dtype='datetime64[ns]', freq='C')
(10, 1)
                  EMA1
Date                  
2023-07-03  440.056946
2023-07-05  436.851807
2023-07-06  437.901001
2023-07-07  435.963165
2023-07-10  440.754120
2023-07-11  438.370239
2023-07-12  438.150208
2023-07-13  439.972931
2023-07-14  437.932007
2023-07-17  438.288666


# Get Prepraed To ingest data into BQ , we have to create dataframe and convert to Json-Rowns

In [74]:
outputDF=pd.DataFrame(data=[ [today,asset_name,prediction_col,dtStr_imported,model_id] ],columns=["prediction_date","asset_name","prediction_name","pred_timestamp","model_id"])
print(outputDF.info())
outputDF

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prediction_date  1 non-null      object
 1   asset_name       1 non-null      object
 2   prediction_name  1 non-null      object
 3   pred_timestamp   1 non-null      object
 4   model_id         1 non-null      object
dtypes: object(5)
memory usage: 168.0+ bytes
None


,prediction_date,asset_name,prediction_name,pred_timestamp,model_id
0,2023-06-30,SPY,EMA1,2023-07-05 08:02:26,spy-ema1-60t10-ds0115t0523


In [75]:
jsonOutput = json.loads(outputDF.to_json(orient = 'records'))
for item in jsonOutput:
    
    dataFeature=dfFeature.reset_index()[[date_col,prediction_col]]
    dataFeature[date_col]=dataFeature[date_col].dt.strftime('%Y-%m-%d')
    dataFeature.columns=["input_date","input_feature"]
    jsonFeature= json.loads(dataFeature.to_json(orient = 'records'))
    item["feature_for_prediction"]=jsonFeature
    
    dataPred=dfPrediction.reset_index()[[date_col,prediction_col]]
    dataPred[date_col]=dataPred[date_col].dt.strftime('%Y-%m-%d')
    dataPred.columns=["output_date","output_value"]
    jsonPred= json.loads(dataPred.to_json(orient = 'records'))
    item["prediction_result"]=jsonPred
 
with open("fin_prediction.json", "w") as outfile:
    json.dump(jsonOutput, outfile)
jsonOutput

[{'prediction_date': '2023-06-30',
  'asset_name': 'SPY',
  'prediction_name': 'EMA1',
  'pred_timestamp': '2023-07-05 08:02:26',
  'model_id': 'spy-ema1-60t10-ds0115t0523',
  'feature_for_prediction': [{'input_date': '2023-04-05',
    'input_feature': 403.7406},
   {'input_date': '2023-04-06', 'input_feature': 404.7314},
   {'input_date': '2023-04-10', 'input_feature': 405.6184},
   {'input_date': '2023-04-11', 'input_feature': 406.3641},
   {'input_date': '2023-04-12', 'input_feature': 406.6707},
   {'input_date': '2023-04-13', 'input_feature': 407.9069},
   {'input_date': '2023-04-14', 'input_feature': 408.7347},
   {'input_date': '2023-04-17', 'input_feature': 409.6812},
   {'input_date': '2023-04-18', 'input_feature': 410.5046},
   {'input_date': '2023-04-19', 'input_feature': 411.1656},
   {'input_date': '2023-04-20', 'input_feature': 411.2955},
   {'input_date': '2023-04-21', 'input_feature': 411.4599},
   {'input_date': '2023-04-24', 'input_feature': 411.6727},
   {'input_date'

# Ingest Data to BigQuery 

In [76]:
try:
    table=client.get_table(table_id)
    print("Table {} already exists.".format(table_id))
    print(table.schema)
except Exception as ex :
    print(str(ex))
#if error  please create table and other configuration as  bq_prediction.txt    

job_config = bigquery.LoadJobConfig(
# schema=[  ]
)

job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND  
#job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
job = client.load_table_from_json(jsonOutput,table_id, job_config = job_config)
if job.errors is not None:
    print(job.error_result)
    print(job.errors)
else:
    print(f"Import to bigquery successfully  {len(jsonOutput)} records")
    
#job_config.schema

Table pongthorn.FinAssetForecast.fin_movement_forecast already exists.
[SchemaField('prediction_result', 'RECORD', 'REPEATED', None, None, (SchemaField('output_value', 'FLOAT', 'NULLABLE', None, None, (), None), SchemaField('output_date', 'DATE', 'NULLABLE', None, None, (), None)), None), SchemaField('prediction_date', 'DATE', 'NULLABLE', None, None, (), None), SchemaField('feature_for_prediction', 'RECORD', 'REPEATED', None, None, (SchemaField('input_feature', 'FLOAT', 'NULLABLE', None, None, (), None), SchemaField('input_date', 'DATE', 'NULLABLE', None, None, (), None)), None), SchemaField('pred_timestamp', 'TIMESTAMP', 'NULLABLE', None, None, (), None), SchemaField('prediction_name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('asset_name', 'STRING', 'NULLABLE', None, None, (), None), SchemaField('model_id', 'STRING', 'NULLABLE', None, None, (), None)]
import to bigquery successfully  1 records


In [132]:
#return   f"The system has done predicting price movement of {asset_name}-{prediction_col}-{today}."